In [27]:
include("spin_chains.jl")
include("vqe.jl")
include("operator.jl")
using LinearAlgebra
using Random

LoadError: LoadError: [91minvalid redefinition of constant Operator[39m
in expression starting at /Users/johnvandyke/postdoc_vt/projects/adapt_vqe_correlated/numerics/bp-mitigation-code/julia/operator.jl:4

In [6]:
L=4;
Jxy=1.0;
Jz=0.5;
PBCs=true;

In [7]:
xxzmat = xxz_matrix(L,Jxy,Jz,PBCs)
evals = eigvals(xxzmat)
println(evals[1])

-6.744562646538029


In [3]:
function two_local_ansatz(n::Int64, d::Int64=2)
    res = Array{String,1}()
    for _=1:d
        for i=1:n
            for j=1:n
                if i<=j continue end
                l = split('I'^n, "")
                l[i] = "X"
                l[j] = "Y"
                s = string(l...)
                push!(res, s)
                l[i] = "Z"
                s = string(l...)
                push!(res, s)
            end
        end
    end
    return res
end

two_local_ansatz (generic function with 2 methods)

In [25]:
map(pauli_string_to_pauli,two_local_ansatz(L, 1))

12-element Array{Pauli{UInt64},1}:
 
 
 
 
 
 
 
 
 
 
 
 

Pauli(x=0100, y=1000, z=0000, ph=+1)Pauli(x=0000, y=1000, z=0100, ph=+1)Pauli(x=0010, y=1000, z=0000, ph=+1)Pauli(x=0000, y=1000, z=0010, ph=+1)Pauli(x=010, y=100, z=000, ph=+1)Pauli(x=000, y=100, z=010, ph=+1)Pauli(x=0001, y=1000, z=0000, ph=+1)Pauli(x=0000, y=1000, z=0001, ph=+1)Pauli(x=001, y=100, z=000, ph=+1)Pauli(x=000, y=100, z=001, ph=+1)Pauli(x=01, y=10, z=00, ph=+1)Pauli(x=00, y=10, z=01, ph=+1)Pauli(x=0100, y=1000, z=0000, ph=+1)Pauli(x=0100, y=1000, z=0000, ph=+1)Pauli(x=0000, y=1000, z=0100, ph=+1)Pauli(x=0000, y=1000, z=0100, ph=+1)Pauli(x=0010, y=1000, z=0000, ph=+1)Pauli(x=0010, y=1000, z=0000, ph=+1)Pauli(x=0000, y=1000, z=0010, ph=+1)Pauli(x=0000, y=1000, z=0010, ph=+1)Pauli(x=010, y=100, z=000, ph=+1)Pauli(x=010, y=100, z=000, ph=+1)Pauli(x=000, y=100, z=010, ph=+1)Pauli(x=000, y=100, z=010, ph=+1)Pauli(x=0001, y=1000, z=0000, ph=+1)Pauli(x=0001, y=1000, z=0000, ph=+1)Pauli(x=0000, y=1000, z=0001, ph=+1)Pauli(x=0000, y=1000, z=0001, ph=+1)Pauli(x=001, y=100, z=000, p

In [26]:
xxzham.paulis

12-element Array{Pauli,1}:
 
 
 
 
 
 
 
 
 
 
 
 

Pauli(x=1100, y=0000, z=0000, ph=+1)Pauli(x=0000, y=1100, z=0000, ph=+1)Pauli(x=0000, y=0000, z=1100, ph=+1)Pauli(x=110, y=000, z=000, ph=+1)Pauli(x=000, y=110, z=000, ph=+1)Pauli(x=000, y=000, z=110, ph=+1)Pauli(x=11, y=00, z=00, ph=+1)Pauli(x=00, y=11, z=00, ph=+1)Pauli(x=00, y=00, z=11, ph=+1)Pauli(x=1001, y=0000, z=0000, ph=+1)Pauli(x=0000, y=1001, z=0000, ph=+1)Pauli(x=0000, y=0000, z=1001, ph=+1)Pauli(x=1100, y=0000, z=0000, ph=+1)Pauli(x=1100, y=0000, z=0000, ph=+1)Pauli(x=0000, y=1100, z=0000, ph=+1)Pauli(x=0000, y=1100, z=0000, ph=+1)Pauli(x=0000, y=0000, z=1100, ph=+1)Pauli(x=0000, y=0000, z=1100, ph=+1)Pauli(x=110, y=000, z=000, ph=+1)Pauli(x=110, y=000, z=000, ph=+1)Pauli(x=000, y=110, z=000, ph=+1)Pauli(x=000, y=110, z=000, ph=+1)Pauli(x=000, y=000, z=110, ph=+1)Pauli(x=000, y=000, z=110, ph=+1)Pauli(x=11, y=00, z=00, ph=+1)Pauli(x=11, y=00, z=00, ph=+1)Pauli(x=00, y=11, z=00, ph=+1)Pauli(x=00, y=11, z=00, ph=+1)Pauli(x=00, y=00, z=11, ph=+1)Pauli(x=00, y=00, z=11, ph=+1)P

In [22]:
xxzham = xxz_model(L,Jxy,Jz,PBCs)

# ansatz_ops = map(pauli_string_to_pauli,["IIXY","XYII","XIYI","XIIY","IXYI","IXIY"]);
# ansatz_ops = map(pauli_string_to_pauli,two_local_ansatz(L, 1));
ansatz_ops = xxzham.paulis

init_state = zeros(ComplexF64,2^L);
init_state[1] = 1.0;

Random.seed!(1234)
rstate = rand!(zeros(ComplexF64,2^L))
rstate = rstate/sqrt(dot(rstate,rstate))
println(dot(rstate,rstate))

tmp_state = zeros(ComplexF64,2^L);
print("init state energy: ",exp_val(xxzham,init_state,tmp_state),'\n')

optimizer = "LN_COBYLA"
init_pt = ones(Float64,length(ansatz_ops));

function maxiter_callback(hist::ADAPTHistory)
    if length(hist.energy) >= 5
        return true
    else
        return false
    end
end

function err_callback(hist::ADAPTHistory)
    if abs(-6.744562646538029 - hist.energy[end]) < 0.01
        return true
    else
        return false
    end
end

adapt_results = adapt_vqe(xxzham,ansatz_ops,L,optimizer,Function[maxiter_callback,err_callback],rstate)

1.0 + 0.0im
init state energy: 2.0 + 0.0im


MethodError: [91mMethodError: no method matching adapt_vqe(::Operator, ::Array{Pauli,1}, ::Int64, ::String, ::Array{Function,1}, ::Array{Complex{Float64},1})[39m
[91m[0mClosest candidates are:[39m
[91m[0m  adapt_vqe(::Operator, [91m::Array{Pauli{T<:Unsigned},1}[39m, ::Int64, ::String, ::Array{Function,N} where N, ::Union{Nothing, Array{Complex{Float64},1}}) where T<:Unsigned at /Users/johnvandyke/postdoc_vt/projects/adapt_vqe_correlated/numerics/bp-mitigation-code/julia/vqe.jl:99[39m
[91m[0m  adapt_vqe(::Operator, [91m::Array{Pauli{T<:Unsigned},1}[39m, ::Int64, ::String, ::Array{Function,N} where N, ::Union{Nothing, Array{Complex{Float64},1}}, [91m::Union{Nothing, Array{Complex{Float64},1}}[39m) where T<:Unsigned at /Users/johnvandyke/postdoc_vt/projects/adapt_vqe_correlated/numerics/bp-mitigation-code/julia/vqe.jl:99[39m
[91m[0m  adapt_vqe(::Operator, [91m::Array{Pauli{T<:Unsigned},1}[39m, ::Int64, ::String, ::Array{Function,N} where N) where T<:Unsigned at /Users

In [13]:
adapt_results.energy

20-element Array{Float64,1}:
  2.569187838203773   
 -0.005100097625580141
 -0.9659905326879719  
 -0.5908077634496991  
 -3.627504696320127   
 -3.8621667926270584  
 -4.0563958113104235  
 -4.859303492672034   
 -5.140488225058578   
 -5.292143660361125   
 -5.153739752366219   
 -5.795145259573981   
 -5.961657704180878   
 -6.025894778424202   
 -6.042123024203386   
 -6.06784351974295    
 -6.053725920275532   
 -6.117560820608681   
 -6.052788405534632   
 -5.867055095674464   

In [19]:
adapt_results.max_grad

3-element Array{Float64,1}:
 1.5701152315936993
 0.766702985209428 
 1.7971734855111003

In [12]:
adapt_results.max_grad_ind

3-element Array{Int64,1}:
 5
 5
 4

In [ ]:
println("energies: ",adapt_results.energy)
# println("energy difference exact - vqe = ",evals[1]-vqe_results[1])

In [13]:
println(xxzham,'\n',ansatz_ops,'\n',opt,'\n', init_pt,'\n', L,'\n', init_state)
vqe_results = VQE(xxzham,ansatz_ops,opt,init_pt,L,init_state)

Operator(Pauli[Pauli(x=1100, y=0000, z=0000, ph=+1), Pauli(x=0000, y=1100, z=0000, ph=+1), Pauli(x=0000, y=0000, z=1100, ph=+1), Pauli(x=110, y=000, z=000, ph=+1), Pauli(x=000, y=110, z=000, ph=+1), Pauli(x=000, y=000, z=110, ph=+1), Pauli(x=11, y=00, z=00, ph=+1), Pauli(x=00, y=11, z=00, ph=+1), Pauli(x=00, y=00, z=11, ph=+1), Pauli(x=1001, y=0000, z=0000, ph=+1), Pauli(x=0000, y=1001, z=0000, ph=+1), Pauli(x=0000, y=0000, z=1001, ph=+1)], Complex{Float64}[1.0+0.0im, 1.0+0.0im, 0.5+0.0im, 1.0+0.0im, 1.0+0.0im, 0.5+0.0im, 1.0+0.0im, 1.0+0.0im, 0.5+0.0im, 1.0+0.0im, 1.0+0.0im, 0.5+0.0im])
Pauli{UInt64}[Pauli(x=10, y=01, z=00, ph=+1), Pauli(x=1000, y=0100, z=0000, ph=+1), Pauli(x=1000, y=0010, z=0000, ph=+1), Pauli(x=1000, y=0001, z=0000, ph=+1), Pauli(x=100, y=010, z=000, ph=+1), Pauli(x=10, y=01, z=00, ph=+1), Pauli(x=1000, y=0100, z=0000, ph=+1), Pauli(x=1000, y=0010, z=0000, ph=+1), Pauli(x=1000, y=0001, z=0000, ph=+1), Pauli(x=100, y=010, z=000, ph=+1)]
Opt(LN_COBYLA, 10)
[1.0, 1.0,

(-6.744562646537241, [2.95962, 2.49109, 1.52576, 0.82503, 1.0931, 1.10728, 0.467375, 0.7854, 0.459662, 0.87976], :ROUNDOFF_LIMITED)

In [18]:
for op in ansatz_ops
    com=commutator(xxzham,Operator([op],[1.0]))
    println(exp_val(com,rstate,tmp_state))
end

-1.249000902703301e-16 - 1.1522231774968592im
-2.7755575615628914e-17 - 0.357659724483542im
5.551115123125783e-17 - 0.6864200491989852im
9.020562075079397e-17 + 1.4073026961179929im
-5.551115123125783e-17 + 1.6099438559163985im
-8.326672684688674e-17 + 0.41797747313422473im
0.0 - 0.8834243975411237im
-5.551115123125783e-17 + 0.9428045672130075im
1.3877787807814457e-17 + 1.3962123467517746im
2.7755575615628914e-17 + 0.7972540600847877im
0.0 - 1.664689408099184im
0.0 + 0.8655557935571071im


In [17]:
p_op = pauli_string_to_pauli("XXXX")
println(p_op)
com=commutator(xxzham,Operator([p_op],[1.0]))
grad = exp_val(com,rstate,tmp_state)
print("XXXX",' ',grad,'\n')

Pauli(x=1111, y=0000, z=0000, ph=+1)
XXXX 0.0 + 0.0im


In [19]:
pvals = "IXYZ"
allplus = (1/sqrt(2^L))*ones(ComplexF64,2^L)
Random.seed!(1234)
rstate = rand!(zeros(ComplexF64,2^L))
rstate = rstate/sqrt(dot(rstate,rstate))
for (a,b,c,d) in Iterators.product(1:4,1:4,1:4,1:4)
    pstr = pvals[a]*pvals[b]*pvals[c]*pvals[d]
    p_op = pauli_string_to_pauli(pstr)
    com=commutator(xxzham,Operator([p_op],[1.0]))
    grad = exp_val(com,rstate,tmp_state)
    print(pstr,' ',grad,'\n')
end

IIII 0.0 + 0.0im
XIII 1.5469850564750835e-17 - 0.0838902747789112im
YIII -5.551115123125783e-17 - 1.004714863876261im
ZIII -1.734723475976807e-18 - 0.4045180294371341im
IXII -2.5855247770106127e-17 + 0.25969835238617095im
XXII 1.214306433183765e-17 + 0.4584646882661378im
YXII -9.71445146547012e-17 - 1.0024714143216054im
ZXII -5.975045583693241e-17 - 0.8877069392801973im
IYII 5.551115123125783e-17 + 1.429134790160656im
XYII -3.469446951953614e-17 + 1.5701152315936988im
YYII -3.209238430557093e-17 - 0.03284802526386732im
ZYII 2.7755575615628914e-17 + 0.08684114073451943im
IZII 3.469446951953614e-18 + 0.3967797745742878im
XZII 2.3729114060586586e-17 + 0.5584256512890586im
YZII -2.7755575615628914e-17 - 0.3576597244835421im
ZZII 0.0 - 0.34563475170402114im
IIXI 0.0 - 0.28278968471129773im
XIXI -2.0816681711721685e-17 - 0.23210482665733911im
YIXI 5.551115123125783e-17 - 0.9389278948482063im
ZIXI -1.3877787807814457e-17 - 0.2300393508532384im
IXXI 3.122502256758253e-17 - 0.041352130540758036